In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
#Use this file path for pc
filepath = r"D:\gradschool\Capstone"
#Use this file path for mac 
# filepath = r"/Users/jacksonjoyner/Desktop/MSBA/Capstone/UseThisData/"
os.chdir(filepath)

In [15]:
playerdf = pd.read_csv("ImpactScoreData.csv")
playerdf = playerdf.rename(columns ={"total cs" : "total_cs"})
playerdf["total_cs"]

0         307.0
1         138.0
2         361.0
3         333.0
4          33.0
          ...  
497559    290.0
497560    202.0
497561    347.0
497562     59.0
497563    222.0
Name: total_cs, Length: 497564, dtype: float64

In [16]:
variables = ["playername",'position','teamname',"kills","deaths",'assists','doublekills','triplekills','quadrakills','pentakills','firstbloodkill'
             ,'firstbloodassist','firstbloodvictim','dpm','wpm','goldspent','total_cs','minionkills','monsterkills',
             'monsterkillsenemyjungle','goldat15','xpat15','csat15','golddiffat15','xpdiffat15',
             'csdiffat15','killsat15','deathsat15','assistsat15','result','impactscore','KDA','KP','D%',"earnedgoldshare",
            'damageshare','cspm','damagetakenperminute']

dfuses = playerdf[variables][playerdf["datacompleteness"]== "complete"].dropna(how = "any")
dfuses.to_csv("nonullimpact.csv",index = False)

#dfuses.to_csv("allyearsnonulls.csv")

From testing, this is the most accurate model I can create

In [17]:
variables = ['kills','deaths','assists','total_cs','earnedgoldshare', 'damageshare','dpm','cspm']
x = dfuses[variables].to_numpy() 
y = dfuses["result"]

In [6]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=0)

In [7]:
# instantiate the model (using the default parameters)
logreg = LogisticRegression(max_iter = 100000)

In [9]:
# fit the model with data
logreg = logreg.fit(x_train,y_train)

#
y_pred=logreg.predict(x_test)
logreg.coef_, logreg.intercept_

(array([[ 6.48588825e-01, -9.33978440e-01,  4.45216842e-01,
         -7.47554526e-03, -3.76493714e+01, -8.53703004e+00,
          2.23681085e-03,  9.85762472e-01]]),
 array([1.88730459]))

In [9]:
logreg.score(x_test,y_test)

0.8922474997701525

In [129]:
y_pred

array([1, 0, 1, ..., 0, 1, 1], dtype=int64)

In [11]:
#https://www.datasklr.com/ols-least-squares-regression/multicollinearity
model = sm.OLS(y_train, sm.add_constant(x_train)).fit()
Y_pred = model.predict(sm.add_constant(x_test))
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                 result   R-squared:                       0.592
Model:                            OLS   Adj. R-squared:                  0.592
Method:                 Least Squares   F-statistic:                 5.323e+04
Date:                Mon, 07 Mar 2022   Prob (F-statistic):               0.00
Time:                        17:41:49   Log-Likelihood:                -81560.
No. Observations:              293671   AIC:                         1.631e+05
Df Residuals:                  293662   BIC:                         1.632e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6919      0.003    213.851      0.0